In [47]:
# import DiCE
import dice_ml
#from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing

In [57]:
import pandas as pd
import io
import requests
names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num']     
url="https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')),names=names,na_values='?')
df['num']=df['num'].map({0:0,1:1,2:1,3:1,4:1})
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [58]:
df.dropna()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,1
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,1
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,1


In [59]:
d = dice_ml.Data(dataframe=df, continuous_features=['age','trestbps','chol','thalach','oldpeak'], outcome_name='num')

In [60]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [61]:
train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'num']
y_train = train.loc[:, train.columns == 'num']

In [62]:
X_train.iloc[0]

age            0.617021
trestbps       0.188679
chol           0.438356
thalach        0.526718
oldpeak        0.709677
sex_0.0        0.000000
sex_1.0        1.000000
cp_1.0         0.000000
cp_2.0         0.000000
cp_3.0         0.000000
cp_4.0         1.000000
fbs_0.0        1.000000
fbs_1.0        0.000000
restecg_0.0    0.000000
restecg_1.0    1.000000
restecg_2.0    0.000000
exang_0.0      1.000000
exang_1.0      0.000000
slope_1.0      0.000000
slope_2.0      0.000000
slope_3.0      1.000000
ca_0.0         0.000000
ca_1.0         0.000000
ca_2.0         0.000000
ca_3.0         1.000000
thal_3.0       0.000000
thal_6.0       1.000000
thal_7.0       0.000000
Name: 285, dtype: float32

In [63]:
ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), activation=tf.nn.relu))#kernel_regularizer=keras.regularizers.l1(0.001)
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=100, verbose=0)#, class_weight={0:1,1:2})
# the training will take some time for 100 epochs.
# you can wait or set verbose=1 to see the progress of training.

In [64]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=ann_model, backend=backend)

In [65]:
df['num'].unique()

array([0, 1])

In [68]:
df.iloc[1]

age          67
sex           1
cp            4
trestbps    160
chol        286
fbs           0
restecg       2
thalach     108
exang         1
oldpeak     1.5
slope         2
ca            3
thal          3
num           1
Name: 1, dtype: object

In [69]:
query_instance = {'age':67, 
                  'sex':1, 
                  'cp':4, 
                  'trestbps':160, 
                  'chol':286,
                  'fbs': 0, 
                  'restecg':2, 
                  'thalach': 108,
                  'exang':1,
                  'oldpeak':1.5,
                  'slope':2,
                  'ca':3,
                  'thal':3}


#second (1) datapoint in df, answer to this query is 1/yes to heart disease

In [70]:
exp = dice_ml.Dice(d,m)

In [71]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=1, desired_class="opposite")

IndexError: list index out of range